In [203]:
import pandas as pd
from datetime import datetime

In [128]:
libre1 = pd.read_csv(r"\Users\cr591\Dropbox\Catherine R\EXTOD 101 study\Libre data\3015 2.csv",  names=[i for i in range(30)])
df = pd.read_csv(r"\Users\cr591\OneDrive - University of Exeter\Desktop\PhD\Projects\Motivate Libre data\LPOOL\Motivate 08T2D_glucose_19-7-2021.csv", names=[i for i in range(30)])

In [224]:
def try_parsing_date(text):
    text = str(text)
    formats = ("%d-%m-%Y %H:%M:%S", "%d-%m-%Y %H:%M:%S", "%d/%m/%Y %H:%M",
               "%d-%m-%Y %H:%M", "%Y-%m-%d %H:%M:%S", "%Y/%m/%d %H:%M:%S",
               "%Y-%m-%d %H:%M", "%Y/%m/%d %H:%M") # add dot format
    for fmt in formats:
        try:
            datetime.strptime(text, fmt)
            return True
        except ValueError:
            pass
    return False

In [234]:
def test_col(col):
    datetime_bool = col.apply(lambda x: try_parsing_date(x))
    if datetime_bool.all() :
            return 'dt'
    try:
        col_num = pd.to_numeric(col)
        if not ((col_num>23) | (col_num<2)).all():
            return 'glc_uk'
        elif not ((col_num>415) | (col_num<36)).all():
            return 'glc_us'
        else:
            return 'unknown'

    except Exception:
        return 'unknown'

In [251]:
def find_header(df):
    dropped = df.dropna()
    dropped.columns = ['time', 'glc']
    count = 0
    for i, row in dropped.iterrows():
        is_date = try_parsing_date(row['time'])
        if not is_date:
            count+=1
            continue
        try:
            float(row['glc'])
            break
        except Exception:
            count +=1
    if count == dropped.shape[0]:
        raise Exception('Problem with input data')
    return dropped.iloc[count:]

In [241]:
def preprocess_data(df, id_colname=None):
    max_rows = df.count().max()
    cols_to_keep = df.count()[df.count()>max_rows*0.7].index
    footer_rows = df[cols_to_keep].iloc[int(-max_rows*0.8):]
    col_type_dict = {'dt':[], 'glc_uk': [], 'glc_us':[]}
    for i in cols_to_keep:
        col_type = test_col(footer_rows[i])
        if col_type != 'unknown':
            col_type_dict[col_type].append(i)
    if (len(col_type_dict['dt'])>0) & (len(col_type_dict['glc_uk'])>0):
        sub_frame = df[[col_type_dict['dt'][0], col_type_dict['glc_uk'][0]]]
        df_processed = find_header(sub_frame)
    elif (len(col_type_dict['dt'])>0) & (len(col_type_dict['glc_us'])>0):
        sub_frame = df[col_type_dict['dt'][0], col_type_dict['glc_us'][0]]
        df_processed = find_header(sub_frame)
        try:
            df_processed['time'] = df_processed['time']/0.0555
        except Exception('Problem with input data')
    else:
        raise Exception('Can\'t identify datetime and/or glucose columns')
    if not id_colname==None:
        df_processed = df_processed.join(df[id_colname], how='left')
    df_processed.reset_index(drop=True, inplace=True)
    return df_processed

In [252]:
preprocess_data(dexcom2)

,time,glc
0,2018-01-17 17:54:12,9.93
1,2018-01-17 17:59:12,10.71
2,2018-01-17 18:04:12,11.10
3,2018-01-17 18:09:12,10.65
4,2018-01-17 18:14:12,11.82
...,...,...
1753,2018-01-23 22:23:48,10.54
1754,2018-01-23 22:28:48,10.16
1755,2018-01-23 22:33:48,8.99
1756,2018-01-23 22:38:48,8.38


In [235]:
preprocess_data(dexcom2)

,time,glc
13,2018-01-17 17:54:12,9.93
14,2018-01-17 17:59:12,10.71
15,2018-01-17 18:04:12,11.10
16,2018-01-17 18:09:12,10.65
17,2018-01-17 18:14:12,11.82
...,...,...
1766,2018-01-23 22:23:48,10.54
1767,2018-01-23 22:28:48,10.16
1768,2018-01-23 22:33:48,8.99
1769,2018-01-23 22:38:48,8.38


In [244]:
dexcom1.head()

,Id,SerialNumber,IsDataBlinded,IsKeepPrivate,GlucoseInternalTime,GlucoseDisplayTime,GlucoseValue,MeterInternalTime,MeterDisplayTime,MeterValue,EventMarkers
0,{EB19A4FA-B976-4D30-A37C-68876AD87B1A},SM64906953,NaN,NaN,2018-04-30 13:47:33,2018-04-30 14:46:43,7.05,2018-04-30 13:44:44,2018-04-30 14:43:54,7.49,NaN
1,{EB19A4FA-B976-4D30-A37C-68876AD87B1A},NaN,NaN,NaN,2018-04-30 13:52:33,2018-04-30 14:51:43,6.99,2018-04-30 13:45:31,2018-04-30 14:44:41,6.88,NaN
2,{EB19A4FA-B976-4D30-A37C-68876AD87B1A},NaN,NaN,NaN,2018-04-30 13:57:33,2018-04-30 14:56:43,6.44,2018-04-30 20:56:44,2018-04-30 21:55:54,6.88,NaN
3,{EB19A4FA-B976-4D30-A37C-68876AD87B1A},NaN,NaN,NaN,2018-04-30 14:02:33,2018-04-30 15:01:43,6.55,2018-05-01 06:33:29,2018-05-01 07:32:38,5.11,NaN
4,{EB19A4FA-B976-4D30-A37C-68876AD87B1A},NaN,NaN,NaN,2018-04-30 14:07:33,2018-04-30 15:06:43,6.27,2018-05-01 20:16:45,2018-05-01 21:15:55,9.82,NaN


In [246]:
preprocess_data(dexcom1)

,time,glc
0,2018-04-30 13:47:33,7.05
1,2018-04-30 13:52:33,6.99
2,2018-04-30 13:57:33,6.44
3,2018-04-30 14:02:33,6.55
4,2018-04-30 14:07:33,6.27
...,...,...
1971,2018-05-07 11:07:13,10.21
1972,2018-05-07 11:12:13,11.27
1973,2018-05-07 11:17:13,11.65
1974,2018-05-07 11:22:14,11.38


In [190]:
test = pd.read_csv('test.csv')
test.dropna(inplace=True)

In [192]:
test_x = pd.read_excel('test_excel.xlsx')
test_x.dropna(inplace=True)

In [240]:
preprocess_data(test, id_colname='ID')

NameError: name 'df_process' is not defined

In [193]:
preprocess_data(test_x)

               ID                time   glc
360  1003_6months 2018-09-21 20:52:00  6.44
365  1003_6months 2018-09-21 20:57:00  7.10
370  1003_6months 2018-09-21 21:02:00  7.05
375  1003_6months 2018-09-21 21:07:00  7.38
380  1003_6months 2018-09-21 21:12:00  7.94
not int
Neither numeric nor datetime
not int
360   2018-09-21 20:52:00
365   2018-09-21 20:57:00
370   2018-09-21 21:02:00
375   2018-09-21 21:07:00
380   2018-09-21 21:12:00
Name: time, dtype: datetime64[ns]
inside range
{'dt': ['time'], 'glc_uk': ['glc'], 'glc_us': []}


,time,glc
0,2018-09-21 14:52:00,9.99
5,2018-09-21 14:57:00,10.16
10,2018-09-21 15:02:00,9.88
15,2018-09-21 15:07:00,9.66
20,2018-09-21 15:12:00,9.16
...,...,...
1772,2019-01-18 20:33:00,5.11
1777,2019-01-18 20:38:00,5.05
1782,2019-01-18 20:43:00,4.99
1787,2019-01-18 20:48:00,4.99


In [144]:
dexcom1.GlucoseValue.describe()

count    1976.000000
mean        8.861483
std         3.376913
min         3.050000
25%         6.550000
50%         8.440000
75%        10.555000
max        20.420000
Name: GlucoseValue, dtype: float64

In [161]:
preprocess_data(df)

object
not int
exception
object
not int
exception
object
not int
object
fine here
object
fine here
inside range
{'dt': [2], 'glc_uk': [4], 'glc_us': []}


,time,glc
3,17-03-2021 11:56,5.5
4,17-03-2021 12:11,5.4
5,17-03-2021 12:26,5.1
6,17-03-2021 12:41,4.8
7,17-03-2021 12:56,5.3
...,...,...
805,25-03-2021 20:26,10.3
806,25-03-2021 20:41,9.9
807,25-03-2021 20:56,9.6
808,25-03-2021 21:11,9.2


In [131]:
libre1.count()

0     8681
1     8665
2     8664
3     8664
4     6202
5      820
6        2
7      178
8       20
9      207
10       1
11       1
12      47
13     199
14     997
15       1
16       1
17       1
18       1
19       0
20       0
21       0
22       0
23       0
24       0
25       0
26       0
27       0
28       0
29       0
dtype: int64

In [133]:
preprocess_data(libre1)

{'dt': [2], 'glc_uk': [4], 'glc_us': []}


,time,glc
4,25-05-2018 19:16,14.9
6,25-05-2018 19:31,13.4
9,25-05-2018 19:47,12.4
13,25-05-2018 20:02,11.9
14,25-05-2018 20:17,11.2
...,...,...
7678,18-08-2018 17:56,3.3
7679,18-08-2018 18:11,3.6
7680,18-08-2018 18:26,6.1
7682,18-08-2018 18:41,8.6


In [43]:
libre2.head(20)

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,Patient report,Generated on,19-07-2021 16:57 UTC,Generated by,Motivate T2D,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Motivate 08 T2D,27-12-1978,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Device,Serial Number,Device Timestamp,Record Type,Historic Glucose mmol/L,Scan Glucose mmol/L,Non-numeric Rapid-Acting Insulin,Rapid-Acting Insulin (units),Non-numeric Food,Carbohydrates (grams),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,FreeStyle Libre Pro,JFMY069-T0689,17-03-2021 11:56,0,5.5,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,FreeStyle Libre Pro,JFMY069-T0689,17-03-2021 12:11,0,5.4,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,FreeStyle Libre Pro,JFMY069-T0689,17-03-2021 12:26,0,5.1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,FreeStyle Libre Pro,JFMY069-T0689,17-03-2021 12:41,0,4.8,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,FreeStyle Libre Pro,JFMY069-T0689,17-03-2021 12:56,0,5.3,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,FreeStyle Libre Pro,JFMY069-T0689,17-03-2021 13:11,0,6.1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,FreeStyle Libre Pro,JFMY069-T0689,17-03-2021 13:26,0,6.4,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
libre1.head()

,Patient report,Generated on,14-09-2018,Generated by,Robert Andrews,EXTOD 101 study,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19
0,3015,27-08-1973,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Meter,Serial Number,Meter Timestamp,Record Type,Historic Glucose(mmol/L),Scan Glucose(mmol/L),Non-numeric Rapid-Acting Insulin,Rapid-Acting Insulin (units),Non-numeric Food,Carbohydrates (grams),Carbohydrates (servings),Non-numeric Long-Acting Insulin,Long-Acting Insulin (units),Notes,Strip Glucose(mmol/L),Ketone(mmol/L),Meal Insulin (units),Correction Insulin (units),User Change Insulin (units),NaN
2,FreeStyle LibreLink,04196E3B-7B98-4306-9C99-A6732EE6A9A9,25-05-2018 19:16,1,NaN,16.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,FreeStyle LibreLink,04196E3B-7B98-4306-9C99-A6732EE6A9A9,25-05-2018 19:16,0,14.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,FreeStyle LibreLink,04196E3B-7B98-4306-9C99-A6732EE6A9A9,25-05-2018 19:29,1,NaN,13.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [134]:
dexcom1 = pd.read_excel(r"\Users\cr591\Dropbox\Catherine R\EXTOD education\Cgms data\2041 V1 PN.xlsx")#,  names=[i for i in range(30)])
dexcom2 = pd.read_excel(r"\Users\cr591\Dropbox\Catherine R\EXTOD education\Cgms data\1029 v1 An.xlsx") #,  names=[i for i in range(30)])

In [135]:
dexcom1.head()

,Id,SerialNumber,IsDataBlinded,IsKeepPrivate,GlucoseInternalTime,GlucoseDisplayTime,GlucoseValue,MeterInternalTime,MeterDisplayTime,MeterValue,EventMarkers
0,{EB19A4FA-B976-4D30-A37C-68876AD87B1A},SM64906953,NaN,NaN,2018-04-30 13:47:33,2018-04-30 14:46:43,7.05,2018-04-30 13:44:44,2018-04-30 14:43:54,7.49,NaN
1,{EB19A4FA-B976-4D30-A37C-68876AD87B1A},NaN,NaN,NaN,2018-04-30 13:52:33,2018-04-30 14:51:43,6.99,2018-04-30 13:45:31,2018-04-30 14:44:41,6.88,NaN
2,{EB19A4FA-B976-4D30-A37C-68876AD87B1A},NaN,NaN,NaN,2018-04-30 13:57:33,2018-04-30 14:56:43,6.44,2018-04-30 20:56:44,2018-04-30 21:55:54,6.88,NaN
3,{EB19A4FA-B976-4D30-A37C-68876AD87B1A},NaN,NaN,NaN,2018-04-30 14:02:33,2018-04-30 15:01:43,6.55,2018-05-01 06:33:29,2018-05-01 07:32:38,5.11,NaN
4,{EB19A4FA-B976-4D30-A37C-68876AD87B1A},NaN,NaN,NaN,2018-04-30 14:07:33,2018-04-30 15:06:43,6.27,2018-05-01 20:16:45,2018-05-01 21:15:55,9.82,NaN
